In [1]:
import numpy as np
import pandas as pd
from gensim.models import FastText
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
# 경고메시지 무시
import warnings
warnings.filterwarnings("ignore")

In [3]:
word_vectors = FastText.load("../Embedding/embedding_data/FastText_embedding.model").wv
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))
positive = model.cluster_centers_[0]
negative = model.cluster_centers_[1]

In [5]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=100, restrict_vocab=None)

[('시시콜콜하다', 0.9790395498275757),
 ('하지만그렇다', 0.9768967628479004),
 ('지만그렇다', 0.9732798933982849),
 ('같다사실', 0.9717927575111389),
 ('말똥말똥하다', 0.971153736114502),
 ('서먹서먹하다', 0.9711471796035767),
 ('왁자지껄하다', 0.970209538936615),
 ('얼떨떨하다', 0.9702087044715881),
 ('둥글둥글하다', 0.9689118266105652),
 ('조마조마하다', 0.9684338569641113),
 ('되다너무', 0.9683095216751099),
 ('고분고분하다', 0.9680849313735962),
 ('좋다항상', 0.9680631756782532),
 ('말다난', 0.9680625796318054),
 ('이야그렇다', 0.9676642417907715),
 ('그렇다모르다', 0.9675673842430115),
 ('좋다너무', 0.9669384956359863),
 ('도그렇다', 0.9669173955917358),
 ('싱숭생숭하다', 0.9667295217514038),
 ('찹찹하다', 0.9666922092437744),
 ('그렇다이제', 0.9666582345962524),
 ('우락부락하다', 0.9664474725723267),
 ('에그렇다', 0.966314971446991),
 ('좋다이렇다', 0.9662700891494751),
 ('구질구질하다', 0.9661862254142761),
 ('같다힘들다', 0.9657034873962402),
 ('좋다생각', 0.9654325842857361),
 ('뿐그렇다', 0.9654044508934021),
 ('알다난', 0.9652741551399231),
 ('보다아니다', 0.965167760848999),
 ('같다결국', 0.965067446231842),
 ('사람그렇다', 0.96

In [6]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=100, restrict_vocab=None)

[('국극사', 0.9958274364471436),
 ('건청궁', 0.9952571392059326),
 ('컨텐더', 0.994756817817688),
 ('생테티엔', 0.994133472442627),
 ('기성복', 0.9940735697746277),
 ('씨프', 0.9937923550605774),
 ('협률사', 0.9928714036941528),
 ('오대산', 0.9926621317863464),
 ('일사', 0.9922834634780884),
 ('묘족', 0.992162823677063),
 ('희현당', 0.9920036792755127),
 ('수궁가', 0.9916847944259644),
 ('손병희', 0.9915728569030762),
 ('향악', 0.9914895296096802),
 ('로타링기아', 0.9914132356643677),
 ('에인헤랴르', 0.9914026260375977),
 ('탄실', 0.991307258605957),
 ('황귀비', 0.9911763668060303),
 ('예로니모', 0.9910240173339844),
 ('소켓', 0.990943431854248),
 ('리어타드', 0.9908573627471924),
 ('이혜경', 0.9907805919647217),
 ('헤르츠스프룽', 0.9906737208366394),
 ('흑막', 0.9906485080718994),
 ('어윈', 0.9905637502670288),
 ('아헨', 0.9905596375465393),
 ('섬서성', 0.9904048442840576),
 ('넵투늄', 0.9904024004936218),
 ('효장태황태후', 0.9903315901756287),
 ('저위비건법', 0.9901962280273438),
 ('지글러', 0.9900305867195129),
 ('동래', 0.9898970127105713),
 ('동고동락', 0.9898486137390137),
 ('심청가', 

In [7]:
words = pd.DataFrame(word_vectors.vocab.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors.wv[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])
words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [32]:
DATA_IN_PATH = '../data/preprocess/'
# 데이터 불러오기 !
KorQuAd = pd.read_csv(DATA_IN_PATH + 'KorQuAd_preprocess_Okt.csv', encoding='UTF8')
naverReview = pd.read_csv(DATA_IN_PATH + 'naverReview_preprocess_Okt.csv', encoding='UTF8')
ultraDiary = pd.read_csv(DATA_IN_PATH + 'ultraDiary_preprocess_Okt.csv', encoding='UTF8')
#데이터를 하나의 데이터 프레임으로 합치자!
data = KorQuAd.append(naverReview)
data = data.append(ultraDiary)
print(data.shape) # 출력 결과 23만 5000개에 가가운 데이터 !
data.head()

(234941, 1)


,context
0,년 바그너 괴테 파우스트 을 처음 읽다 그 내용 에 마음 끌리다 이르다 소재 로 하...
1,한편 년 부터 바그너 와 알 고 지내다 리스트 잊혀지다 있다 악장 을 부활 시키다 ...
2,작품 라단조 아주 신중하다 박자 부드럽다 서주 로 서주 로 시작 되다 여기 에는 주...
3,커닐링구스커닐 링거 스 쿤닐링구스 영어 늘다 입술 혀 입 모든 구강 기관 으로 여성...
4,일반 적 인 통계 에 따르다 여성 가다 오르가슴 을 얻다 위해 직접 적 인 음핵 자...


In [35]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
tfidf.fit(data)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(data)

In [37]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.title.split()))

In [38]:

def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out
  